In [1]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

In [2]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

path_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/path_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(path_dbs))

336


In [3]:
from random import sample
from sklearn.model_selection import train_test_split

sample_20 = sample(path_dbs, 20)
train, test = train_test_split(sample_20, train_size=.5, shuffle=True, random_state=rand)
print("train %d, test %d" %(len(train), len(test)))

train 10, test 10


corrida de sanity check con path

In [4]:
import tempfile
res_dir = tempfile.mkdtemp()
print("using %s for 20 run" % (res_dir))

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="path",
    train=train,
    test=test,
    docker_client=docker_client,
)

INFO:experiments.all:train_and_evaluate with train size 10, test size 10
INFO:experiments.all:Running training
INFO:scripts.docker:creating /tmp/tmpitud061x dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing


using /tmp/tmpitud061x for 20 run


INFO:scripts.docker:running at container 08e1c9e9f1aee691c61e7140067623b7fa7ed29c21a0991deccc3541a9a3e096. Use `docker logs 08e1c9e9f1aee691c61e7140067623b7fa7ed29c21a0991deccc3541a9a3e096 --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /tmp/tmpitud061x/*/reprScores.txt
INFO:misc.combinescores:working on 8 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /tmp/tmpitud061x/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating tianxiangbing - mobile-select-area
INFO:scripts.evaluate:Evaluating alphagov - webchat-prototype
INFO:scripts.evaluate:Evaluating ckeditor - ckeditor5-dev
INFO:scripts.evaluate:Evaluating Klemek - snex.io
INFO:scripts.evaluate:Evaluating cdaniel - wardleymapstool
INFO:scripts.evaluate:Evaluating apache - couchdb-fauxton
INFO:scripts.evaluate:Evaluating k88hudson - w

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,1,0,3,1,1.0


dio pésimo denuevo con pocas dbs, nada como nosql que con poquitas da bien. Probando con 50/50.

In [5]:
sample_100 = sample(path_dbs, 100)
train, test = train_test_split(sample_100, train_size=.5, shuffle=True, random_state=rand)
print("train %d, test %d" %(len(train), len(test)))

train 50, test 50


In [6]:
import tempfile
res_dir = tempfile.mkdtemp()
print("using %s for 100 run" % (res_dir))

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="path",
    train=train,
    test=test,
    docker_client=docker_client,
)

INFO:experiments.all:train_and_evaluate with train size 50, test size 50
INFO:experiments.all:Running training
INFO:scripts.docker:creating /tmp/tmp2e76gonr dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing


using /tmp/tmp2e76gonr for 100 run


INFO:scripts.docker:running at container 833e070e0cf2e3cbfc52736bc88d7cd1cce869f7d528cb45cdc872131c382686. Use `docker logs 833e070e0cf2e3cbfc52736bc88d7cd1cce869f7d528cb45cdc872131c382686 --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /tmp/tmp2e76gonr/*/reprScores.txt
INFO:misc.combinescores:working on 39 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /tmp/tmp2e76gonr/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating mgh3326 - DoItNodeJS
INFO:scripts.evaluate:Evaluating YellowLabTools - YellowLabTools
INFO:scripts.evaluate:Evaluating prebid - prebid-js-build-generator
INFO:scripts.evaluate:Evaluating firephp - firephp-extension
INFO:scripts.evaluate:Evaluating gilbitron - Raneto
INFO:scripts.evaluate:Evaluating linagora - hublin
INFO:scripts.evaluate:Evaluating assaf - zombie
I

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.010363,0.153846,0.009804,13,2,191,10,1.3


haciendo una prueba de tomar muchos cortes para ver si todos dan mal

In [7]:
from sklearn.model_selection import KFold
import numpy as np

sample_30 = sample(path_dbs, 30)
# fixing random state to make this repeatable
kf = KFold(n_splits=3)
collected_results = []
base_results_dir = "/home/pablo/dev-results/path-30-3fold-"
for i, (train_index, test_index) in enumerate(kf.split(sample_30)):
    print("Fold %d" % (i))
    res_dir = "%s%d" % (base_results_dir, i)
    train_split = np.take(sample_30, train_index)
    test_split = np.take(sample_30, test_index)

    scores = train_and_evaluate(
        config=config,
        results_dir=res_dir,
        query_type="path",
        train=train_split.tolist(),
        test=test_split.tolist(),
        docker_client=docker_client,
    )
    print(scores)
    collected_results.append(scores)
print("done")

INFO:experiments.all:train_and_evaluate with train size 20, test size 10
INFO:experiments.all:Running training
INFO:scripts.docker:creating /home/pablo/dev-results/path-30-3fold-0 dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing


Fold 0


INFO:scripts.docker:running at container b600026092940ee8102e4af9fc349f6ad6599bea76b26947c43099a8b09f42cf. Use `docker logs b600026092940ee8102e4af9fc349f6ad6599bea76b26947c43099a8b09f42cf --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/path-30-3fold-0/*/reprScores.txt
INFO:misc.combinescores:working on 17 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/path-30-3fold-0/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Binomi0 - crm-backoffice-api
INFO:scripts.evaluate:Evaluating tianxiangbing - mobile-select-area
INFO:scripts.evaluate:Evaluating ministryofjustice - iis
INFO:scripts.evaluate:Evaluating microsoft - azpipelines-kubernetesUI-WebApp
INFO:scripts.evaluate:Evaluating airontsang - up-up
INFO:scripts.evaluate:Evaluating MgBac

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0   0.018519     1.0  0.018519                        1                 1   

   suprious alerts  projects with atr  avg atr per proj  
0               53                  1               1.0  
Fold 1


INFO:scripts.docker:running at container 80df20a37f57f975d3fc03bad5fe0febb057d6b3ac71933d1842420724df1747. Use `docker logs 80df20a37f57f975d3fc03bad5fe0febb057d6b3ac71933d1842420724df1747 --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/path-30-3fold-1/*/reprScores.txt
INFO:misc.combinescores:working on 16 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/path-30-3fold-1/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating larrycameron80 - mvp-product
INFO:scripts.evaluate:Evaluating GoogleChromeLabs - ui-element-samples
INFO:scripts.evaluate:Evaluating GoogleChromeLabs - sw-precache
INFO:scripts.evaluate:Evaluating sxfshdf - server-js
INFO:scripts.evaluate:Evaluating simpletut - Universal-React-Apollo-Registration
INFO:scripts.evaluate:E

   precision    recall  accuracy  alerts to recover (atr)  alerts recovered  \
0   0.666667  0.545455  0.428571                       11                 6   

   suprious alerts  projects with atr  avg atr per proj  
0                3                  3          3.666667  
Fold 2


INFO:scripts.docker:running at container 251bdbafc945758446aeb3243c5f8af5e68adf9d3e4d81efef0e40311ee0169b. Use `docker logs 251bdbafc945758446aeb3243c5f8af5e68adf9d3e4d81efef0e40311ee0169b --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/path-30-3fold-2/*/reprScores.txt
INFO:misc.combinescores:working on 17 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/path-30-3fold-2/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating eleboucher - faceapp-express-example
INFO:scripts.evaluate:Evaluating davglass - yui-local
INFO:scripts.evaluate:Evaluating henszey - etcd-browser
INFO:scripts.evaluate:Evaluating michalbluj - CeasarsOffers
INFO:scripts.evaluate:Evaluating Alfarie - LinkitSmart
INFO:scripts.evaluate:Evaluating FabricLabs - maki
INFO:scr

   precision    recall  accuracy  alerts to recover (atr)  alerts recovered  \
0   0.076923  0.666667  0.074074                        3                 2   

   suprious alerts  projects with atr  avg atr per proj  
0               24                  2               1.5  
done


In [8]:
import pandas as pd

concated_results = pd.concat(collected_results)
pd.concat([concated_results, concated_results.apply(['mean'])])

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.018519,1.000000,0.018519,1.0,1.0,53.000000,1.0,1.000000
0,0.666667,0.545455,0.428571,11.0,6.0,3.000000,3.0,3.666667
0,0.076923,0.666667,0.074074,3.0,2.0,24.000000,2.0,1.500000
mean,0.254036,0.737374,0.173721,5.0,3.0,26.666667,2.0,2.055556
